In [1]:
from sklearn.decomposition import PCA

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix

In [2]:
df=pd.read_excel('../Data/default of credit card clients.xls', header=1)

In [4]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [5]:
df=df.iloc[:,1:]

In [6]:
# df.info()

In [7]:
x=df.iloc[:,:-1]

In [8]:
y=df['default payment next month']

In [9]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(x)
x_scaled=scaler.transform(x)

In [10]:
pca=PCA()

In [11]:
# 지정된 데이터에서 주성분 추출
pca.fit(x_scaled)

PCA()

In [12]:
# 주성분에 투영후 분산 정도 설명(비율)
pca.explained_variance_ratio_

array([0.28448215, 0.17818817, 0.06743307, 0.06401154, 0.04457556,
       0.04161737, 0.03946035, 0.03859201, 0.03788041, 0.03404042,
       0.03186036, 0.02968788, 0.02482385, 0.02279956, 0.01754959,
       0.01129943, 0.01083167, 0.00820396, 0.00572984, 0.00305025,
       0.00177324, 0.00109979, 0.00100953])

In [14]:
cumsum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(cumsum>=0.95)+1

print(f'주성분 갯수(차원): {d}')

주성분 갯수(차원): 15


In [15]:
# 지정된 PC 갯수로 주성분 데이터 압축/융합
pca=PCA(n_components=d)

In [16]:
pca.fit(x_scaled)

PCA(n_components=15)

In [17]:
crdPCA=pca.transform(x_scaled)

In [18]:
crdPCA.shape

(30000, 15)

In [19]:
# ## 주성분과 데이터 투영 분산 정도 시각화
# plt.plot(cumsum)
# plt.plot(range(0,23), cumsum, 'ro')
# # plt.plot(d, 0.95, 'ro')
# # plt.plot([15,15],[0,0.95], 'b--')
# plt.plot([0,20],[0.95,0.95], 'b--')
# plt.axis([0,20,0,1])
# plt.xlabel('Dimention')
# plt.ylabel('Explained Variance')
# plt.show()

In [22]:
type(crdPCA)

numpy.ndarray

In [21]:
y=np.array(y)

In [23]:
y=y.reshape(-1,1)

In [25]:
y=y.toarray()

In [47]:
crdPCA.shape

(30000, 15)

In [24]:
# 타겟 라벨 -> 원핫인코딩으로 변환
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()
encoder.fit(y.reshape(-1,1))
y=encoder.transform(y.reshape(-1,1))

In [20]:
from keras import Sequential
from keras.layers import Dense

In [78]:
def get_model(d1, d2, d3, learning_rate, n, n1):
        
    model=Sequential()
    model.add(Dense(d1, activation='relu', input_shape=(15,)))
    
    for i in range(n):
        model.add(Dense(n1, activation='relu'))
    
    model.add(Dense(d2, activation='relu'))
    model.add(Dense(d3, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
              
    return model

In [80]:
from scikeras.wrappers import KerasClassifier
clf = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    d1=50,
    d2=30,
    d3=30,
    learning_rate=0.1,
    n=3,
    n1=10
    )

In [81]:
clf.fit(crdPCA, y)

938/938 [==============================] - 3s 2ms/step - loss: 0.4701


KerasClassifier(
	model=<function get_model at 0x0000027B512D9940>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=binary_crossentropy
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	d1=50
	d2=30
	d3=30
	learning_rate=0.1
	n=3
	n1=10
	class_weight=None
)

In [82]:
from sklearn.model_selection import GridSearchCV


params = {

    # 딕셔너리형태로 그리드서치할 인수 입력
    "d1" : [30, 50, 100],
    "d2" : [30, 50],
    "n" : [1,2,3,4],
    'n1' : [30, 50],
    "d3" : [30, 50, 100],
    "optimizer": ["adam"],
    "learning_rate":[0.0001,0.001,0.01,0.1,1]
    
}
gs = GridSearchCV(clf, params, refit=False, cv=3, scoring='accuracy')

gs.fit(crdPCA, y, epochs=3)
print(gs.best_score_, gs.best_params_)

Epoch 1/3
625/625 [==============================] - 2s 2ms/step - loss: 0.4730
Epoch 2/3
625/625 [==============================] - 1s 2ms/step - loss: 0.4335
Epoch 3/3
313/313 [==============================] - 1s 1ms/step
Epoch 1/3
625/625 [==============================] - 1s 2ms/step - loss: 0.4780
Epoch 2/3
625/625 [==============================] - 1s 2ms/step - loss: 0.4405
Epoch 3/3
313/313 [==============================] - 1s 1ms/step
Epoch 1/3
625/625 [==============================] - 2s 2ms/step - loss: 0.4868
Epoch 2/3
625/625 [==============================] - 1s 2ms/step - loss: 0.4531
Epoch 3/3
313/313 [==============================] - 0s 1ms/step
Epoch 1/3
625/625 [==============================] - 1s 2ms/step - loss: 0.4604
Epoch 2/3
625/625 [==============================] - 1s 2ms/step - loss: 0.4327
Epoch 3/3
313/313 [==============================] - 1s 1ms/step
Epoch 1/3
625/625 [==============================] - 2s 2ms/step - loss: 0.4675
Epoch 2/3
625/625 [=

In [59]:
# {'d1': 50, 'd2': 30, 'd3': 30}
# {'d1': 50, 'optimizer': 'adam'}
# {'learning_rate': 0.1}
# {'d2': 50, 'n': 4, 'optimizer': 'adam'}

In [83]:
model2=Sequential()
model2.add(Dense(30, activation='relu', input_shape=(15,)))

for i in range(1):
    model2.add(Dense(30, activation='relu'))

model2.add(Dense(30, activation='relu'))
model2.add(Dense(30, activation='relu'))
model2.add(Dense(2,activation='sigmoid'))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [85]:
model2.fit(crdPCA, y, epochs=10, validation_split=0.2)

Epoch 1/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4282 - accuracy: 0.8193 - val_loss: 0.4136 - val_accuracy: 0.8295
Epoch 2/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4263 - accuracy: 0.8198 - val_loss: 0.4139 - val_accuracy: 0.8295
Epoch 3/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4255 - accuracy: 0.8202 - val_loss: 0.4169 - val_accuracy: 0.8280
Epoch 4/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4240 - accuracy: 0.8191 - val_loss: 0.4134 - val_accuracy: 0.8285
Epoch 5/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4242 - accuracy: 0.8207 - val_loss: 0.4139 - val_accuracy: 0.8290
Epoch 6/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4231 - accuracy: 0.8206 - val_loss: 0.4128 - val_accuracy: 0.8302
Epoch 7/10
750/750 [==============================] - 2s 3ms/step - loss: 0.4220 - accuracy: 0.8214 - val_loss: 0.4143 - val_accuracy: 0.8272
Epoch 